In [1]:
import azureml.core


# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.7.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ANTQYUJ64 to authenticate.
Interactive authentication successfully completed.
hakk-ai
hakk.ai
eastus
b738af92-69d2-4390-a1d8-579dc094b72e


In [3]:
from azureml.core.model import Model

model = Model.register(model_path="Model/xgbIvan.pkl",
                       model_name="finalized_model",
                       workspace=ws)

print('Name:', model.name)
print('Version:', model.version)

Registering model finalized_model
Name: finalized_model
Version: 2


In [5]:
from azureml.core.conda_dependencies import CondaDependencies 


# Usually a good idea to choose specific version numbers
# so training is made on same packages as scoring
myenv = CondaDependencies.create(
    conda_packages=[
        'numpy==1.15.4',
        'scikit-learn==0.19.1', 
        'pandas==0.23.4',        
        'joblib',
        'xgboost'
    ],
    pip_packages=[
        'azureml-defaults>=1.0.45',
        'inference-schema[numpy-support]',
        'inference-schema[pandas-support]'
    ], 
    python_version='3.6.2')

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [9]:
%%writefile score.py
import json
import numpy as np
import joblib
from azureml.core.model import Model
import xgboost
import pandas as pd

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def haversine_km(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

def haversine_m(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 3956 * c
    return m

def preprocessing(data):
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
    data['timestamp'] = pd.DatetimeIndex(data.timestamp)

    data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]), 1, 0)
    data['is_weekday'] = np.where(data['day_of_week'].isin([5,6]), 0, 1)

    data['is_wee_hours'] = np.where(data['hour_of_day'].isin([17,18,19,20,21]), 1, 0)

    data['is_rush_hours_morning'] = np.where(data.timestamp.dt.strftime('%H:%M:%S').between('11:30:00', '01:30:00'), 1, 0)
    data['is_rush_hours_evening'] = np.where(data.timestamp.dt.strftime('%H:%M:%S').between('09:00:00', '12:00:00'), 1, 0)

    data['sin_hour_of_day'] = np.sin(2*np.pi*data.hour_of_day/24)
    data['cos_hour_of_day'] = np.cos(2*np.pi*data.hour_of_day/24)
    data['sin_day_of_week'] = np.sin(2*np.pi*data.day_of_week/7)
    data['cos_day_of_week'] = np.cos(2*np.pi*data.day_of_week/7)


    data['haversine_km'] = haversine_km(data['longitude_origin'], data['latitude_origin'], 
                                 data['longitude_destination'], data['latitude_destination'])

    data['haversine_m'] = haversine_m(data['longitude_origin'], data['latitude_origin'], 
                                 data['longitude_destination'], data['latitude_destination'])


    from sklearn.decomposition import PCA
    coords = np.vstack((data[['latitude_origin', 'longitude_origin']].values,
                    data[['latitude_destination', 'longitude_destination']].values))
    
    pca = PCA().fit(coords)

    data['pickup_pca0'] = pca.transform(data[['latitude_origin', 'longitude_origin']])[:, 0]
    data['pickup_pca1'] = pca.transform(data[['latitude_origin', 'longitude_origin']])[:, 1]
    data['dropoff_pca0'] = pca.transform(data[['latitude_destination', 'longitude_destination']])[:, 0]
    data['dropoff_pca1'] = pca.transform(data[['latitude_destination', 'longitude_destination']])[:, 1]

    data = data.drop(['timestamp'], axis=1)
    
    return data

def init():
    global model

    model_path = Model.get_model_path(model_name='finalized_model')
    
    model = joblib.load(model_path)

    
input_sample = pd.DataFrame(data=[{
    "latitude_origin": -6.141255,
    "longitude_origin": 106.692710,
    "latitude_destination": -6.141150,
    "longitude_destination": 106.693154,
    "timestamp": 1590487113,
    "hour_of_day": 9,
    "day_of_week": 1
}])

# This is an integer type sample. Use the data type that reflects the expected result.
output_sample = np.array([360.00])

# To indicate that we support a variable length of data input,
# set enforce_shape=False
@input_schema('data', PandasParameterType(input_sample, enforce_shape=False))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        #preprocessing
        data = preprocessing(data)
        #result
        result = model.predict(data)    
        return result.tolist()
    
    except Exception as e:
        result = str(e)
        return result

Overwriting score.py


# Deploy locally

In [8]:
from azureml.core import Webservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import LocalWebservice

service_name = 'hakkai-service-test'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

deployment_config = LocalWebservice.deploy_configuration(port=8890)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

print(service.port)

Generating Docker build context.
2020/06/19 20:22:48 Downloading source code...
2020/06/19 20:22:49 Finished downloading source code
2020/06/19 20:22:50 Creating Docker network: acb_default_network, driver: 'bridge'
2020/06/19 20:22:50 Successfully set up Docker network: acb_default_network
2020/06/19 20:22:50 Setting up Docker configuration...
2020/06/19 20:22:51 Successfully set up Docker configuration
2020/06/19 20:22:51 Logging in to registry: hakkai6fb3fa5e.azurecr.io
2020/06/19 20:22:52 Successfully logged into hakkai6fb3fa5e.azurecr.io
2020/06/19 20:22:52 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/06/19 20:22:52 Scanning for dependencies...
2020/06/19 20:22:53 Successfully scanned dependencies
2020/06/19 20:22:53 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB
Step 1/15 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a7695cbba5638933

  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66713 sha256=87a23874afc124771170dfa29d38d610174e3e90f4521d752baac7e33b8954ac
  Stored in directory: /root/.cache/pip/wheels/94/0f/ec/66085641573800014bb0c8b657f3366eff641c42df79abbfe9
  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=78912 sha256=3f32bb1608adcc646b40dba1d34ed14f7683079f1f7ccf6661deab9cdd857870
  Stored in directory: /root/.cache/pip/wheels/02/49/cf/660924cd9bc5fcddc3a0246fe39800c83028d3ccea244de352
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13333 sha256=7c059f19115a96e0ef26e7bcf050d1705aafba6c12961e33f64b5e7e10a3c4b7
  Stored in directory: /root/.cache/pip/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10503 sha256=f9c0a352a7d8d8b76f60fb22fb9c0038275a413bfe328994dfe631ff1bbf7c45
  Stored in directory: /root/.cache/pip/wheels/21/5c/83/1dd7e8a232d

Step 2/5 : COPY azureml-app /var/azureml-app
 ---> f67294499af1
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImI3MzhhZjkyLTY5ZDItNDM5MC1hMWQ4LTU3OWRjMDk0YjcyZSIsInJlc291cmNlR3JvdXBOYW1lIjoiaGFray5haSIsImFjY291bnROYW1lIjoiaGFray1haSIsIndvcmtzcGFjZUlkIjoiYTkxYzQ5YjUtN2RlMC00Yzg4LTk3ZDAtOTdiNjVjMzgwODE3In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in bb98c71e0bff
 ---> dc844fe62b11
Step 4/5 : RUN mv '/var/azureml-app/tmp1kfl1dm1.py' /var/azureml-app/main.py
 ---> Running in 60c0e2a09ea8
 ---> 9f08c3c2c5cc
Step 5/5 : CMD ["runsvdir","/var/runit"]
 ---> Running in ee8ae5869176
 ---> 2db0144a3579
Successfully built 2db0144a3579
Successfully tagged hakkai-service-test:latest
Starting Docker container...
Docker container running.
Checking container health...


ERROR - Error: Container has crashed. Did your init method fail?




Container Logs:
2020-06-19T20:31:52,707103035+00:00 - iot-server/run 
2020-06-19T20:31:52,714480349+00:00 - gunicorn/run 
2020-06-19T20:31:52,715569351+00:00 - rsyslog/run 
2020-06-19T20:31:52,718233557+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a0677afa4eacaa88546a43bc78063d41/lib/libssl.so.1.0.0: no version information available (required by /usr/sbi

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
print(service.get_logs())

In [ ]:
print(service.state)

# Deploy on cloud

In [ ]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'finalized-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass


myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=3, memory_gb=15, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

Running.................................................................................................

In [ ]:
print(service.get_logs())

In [ ]:
print(service.swagger_uri)

In [ ]:
print(service.scoring_uri)

# Test REST API

In [109]:
import requests
import json
import pandas as pd

headers = {'Content-Type': 'application/json'}

if service.auth_enabled:
    headers['Authorization'] = 'Bearer '+service.get_keys()[0]
elif service.token_auth_enabled:
    headers['Authorization'] = 'Bearer '+service.get_token()[0]

print(headers)

test_sample = json.dumps({'data': [
    [1, 2, 3, 4, 5, 6, 7],
    [10, 9, 8, 7, 6, 5, 4]
]})

print(test_sample)

response = requests.post(
    service.scoring_uri,
    data=test_sample, 
    headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())


{'Content-Type': 'application/json'}
{"data": [[1, 2, 3, 4, 5, 6, 7], [10, 9, 8, 7, 6, 5, 4]]}
200
0:00:00.101601
[1318.6348949230166, 1542.3829575282678]


In [108]:
import pandas as pd
import numpy as np

PATH = "Preprocessed_Dataset/SG_car.csv"

data = pd.read_csv(PATH)

data = data.drop(['Unnamed: 0', 'trj_id', 'osname', 
                  'pingtimestamp', 'timestamp', 'rawlat', 
                  'rawlng', 'speed', 'bearing', 
                  'accuracy', 'pickup_time', 'dropoff_time'], axis=1)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

train, test = train_test_split(data, test_size=0.5, train_size = 0.5)
validate, test = train_test_split(test, test_size = 0.3, train_size = 0.7)
print(len(train))
print(len(validate))
print(len(test))

X_train = train[train.columns.difference(['travel_time'])]
Y_train = train["travel_time"]

X_val = validate[validate.columns.difference(['travel_time'])]
Y_val = validate["travel_time"]

X_test = test[test.columns.difference(['travel_time'])]
Y_test = test["travel_time"]



14000
9800
4200


In [132]:
testing = X_test.to_json()
test_sample = json.dumps({'data': [testing]})
y = json.loads(test_sample)


In [137]:
y['data']

['{"cos_pickup_day":{"24841":0.6234898019,"9612":0.6234898019,"14369":-0.222520934,"12047":-0.222520934,"25656":-0.222520934,"8454":-0.9009688679,"11329":-0.9009688679,"24073":0.6234898019,"18420":0.6234898019,"10471":0.6234898019,"26509":-0.222520934,"1278":-0.222520934,"24004":-0.9009688679,"21436":-0.222520934,"7599":-0.9009688679,"21515":0.6234898019,"3026":-0.9009688679,"27154":-0.9009688679,"7319":-0.222520934,"8716":-0.9009688679,"8983":-0.9009688679,"17064":-0.9009688679,"12889":0.6234898019,"17060":-0.9009688679,"25828":-0.222520934,"21064":1.0,"22849":-0.9009688679,"9096":-0.222520934,"1106":0.6234898019,"26349":-0.9009688679,"5820":-0.9009688679,"20702":-0.9009688679,"21966":-0.9009688679,"12879":1.0,"23822":-0.9009688679,"26331":-0.9009688679,"4952":-0.9009688679,"8145":0.6234898019,"9631":0.6234898019,"27741":-0.222520934,"25376":0.6234898019,"26051":-0.9009688679,"17781":-0.9009688679,"27604":-0.222520934,"1320":-0.222520934,"24858":-0.9009688679,"18770":0.6234898019,"853

In [112]:
response = requests.post(
    service.scoring_uri, data=testing, headers=headers)

print(response.status_code)
print(response.elapsed)
print(response.json())

mse = mean_squared_error(Y_test,response.json())
print("RMSE: %.2f" % np.sqrt(mse))

200
0:00:00.142989
'data'


ValueError: Found input variables with inconsistent numbers of samples: [4200, 6]

In [90]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'hakkai-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass


inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

KeyboardInterrupt: 